In [148]:
import numpy as np
import math
import scipy.stats
import random

n = 50

def get_psi():
    return 2*random.uniform(0, 1) - 1

sample = []
for i in range(n):
    elem = [get_psi() for j in range(5)]
    a = 2 + 3*elem[0] - 2*elem[1] + elem[2] + elem[3] - elem[4] 
    elem.append(scipy.stats.norm.rvs(loc=a, scale=1.5, size=1)[0])
    sample.append(elem)

print('Выборка:')
print(np.array(sample))


Выборка:
[[-0.06280446 -0.68057546  0.82655915  0.21755855  0.9552378   1.22720527]
 [-0.38555916 -0.55298734 -0.45712263  0.18334655  0.92821495 -1.17296106]
 [ 0.93627161 -0.55524752  0.73796815 -0.3155274   0.08227209  5.89152657]
 [-0.70583308  0.43413843  0.14451648  0.67871392 -0.20434569 -0.41970936]
 [-0.7492125  -0.66903537  0.14707233  0.83332876  0.86355642  0.0836582 ]
 [-0.86284308 -0.84783281  0.7592978  -0.37269207  0.8322605   0.64565853]
 [ 0.76863394  0.54135854 -0.85633052 -0.50630417 -0.457991    0.58326972]
 [ 0.03229074 -0.15535771 -0.95761825 -0.50792322 -0.33815071  1.27710976]
 [ 0.51760707 -0.7680835  -0.54540421 -0.58647775 -0.69985176  3.13481939]
 [-0.75441327  0.09645254  0.89743255  0.76376208  0.3666464   2.23676158]
 [-0.01245819  0.35091423  0.37302413  0.91182332 -0.95186607  0.6738121 ]
 [ 0.29009882 -0.23174165 -0.81840331 -0.66860919 -0.66477485  1.95138106]
 [ 0.50705406  0.12773019  0.87324491  0.65404491 -0.95542558  6.60381377]
 [ 0.81449223  0

# a) Проверить ксишки на мультиколлениарность

In [149]:
def TSS(y):
    y_average = sum(y)/len(y)
    res = 0
    for i in range(len(y)):
        res += (y[i] - y_average)**2
    return res

def RSS(e):
    return np.matmul(e.transpose(), e)

def R_2(tss, rss):
    return (tss - rss)/tss

def calc_koefs(ksi, y):
    ksi_T = ksi.transpose()
    F = np.matmul(ksi_T, ksi)
    F_inv = np.linalg.inv(F)
    koefs = np.matmul(F_inv, np.matmul(ksi_T, y))
    return koefs

def calc_e(ksi, y, koefs):
    return y - np.matmul(ksi, koefs)

ksi_list = []
y_list = []
for i in range(5):
    elem_ksi = []
    elem_y = []
    for j in range(n):
        line = [1] + sample[j][0 : i] + sample[j][i+1 : 5]
        elem_ksi.append(line)
        elem_y.append(sample[j][i])
    y_list.append(np.array(elem_y))
    ksi_list.append(np.array(elem_ksi))
    
koefs_list = [calc_koefs(ksi_list[i], y_list[i]) for i in range(5)]
e_list = [calc_e(ksi_list[i], y_list[i], koefs_list[i]) for i in range(5)]
RSS_list = [RSS(e_list[i]) for i in range(5)]
TSS_list = [TSS(y_list[i]) for i in range(5)]
R_2_list = [R_2(TSS_list[i], RSS_list[i]) for i in range(5)]

is_multikol = False
for i in range(5):
    print(f'При кси{i+1}=f(от остальных кси)    R^2 = {R_2_list[i]}')
    if R_2_list[i] >= 0.7:
        is_multikol = True
        
if is_multikol:
    print('\n Ксишки мультиколлениарны')
else:
    print('\n Ксишки не являются мультиколлениарными')

При кси1=f(от остальных кси)    R^2 = 0.0651043728647124
При кси2=f(от остальных кси)    R^2 = 0.05473363897740022
При кси3=f(от остальных кси)    R^2 = 0.028109428110884502
При кси4=f(от остальных кси)    R^2 = 0.09491866079327754
При кси5=f(от остальных кси)    R^2 = 0.13139793306866482

 Ксишки не являются мультиколлениарными


# б) Определить уравнение линейной регрессии для этты и проверить значимость коэффициентов

In [150]:
ksi = []
y = []
for i in range(n):
    ksi.append([1] + sample[i][:-1])
    y.append(sample[i][5])
ksi = np.array(ksi)
y = np.array(y)
koefs = calc_koefs(ksi, y)

print('Коэффициенты: \n')
for i in range(len(koefs)):
    print(f'b{i} = {koefs[i]}')

Коэффициенты: 

b0 = 1.661881335371115
b1 = 2.867762438761124
b2 = -1.8376730002399868
b3 = 1.0343743496873399
b4 = 1.0664332857900598
b5 = -0.43155038414924046


In [151]:
alpha = 0.05

def calc_delta(ksi, koefs, i, rss):
    ksi_T = ksi.transpose()
    F = np.matmul(ksi_T, ksi)
    F_inv = np.linalg.inv(F)
    p = ksi.shape[1]
    return koefs[i]*math.sqrt(n-p)/math.sqrt(rss*F_inv[i][i])

def integrate_student(a, b, N):
    def student(x):
        return math.gamma((N+1)/2)/(math.sqrt(math.pi*N)*math.gamma(N/2)*(1+x**2/N)**((N+1)/2))
    return scipy.integrate.quad(student, a, b)[0]
    
e = calc_e(ksi, y, koefs)
rss = RSS(e)
    
deltas = [calc_delta(ksi, koefs, i, rss) for i in range(6)]

p_values = [2*integrate_student(abs(deltas[i]), math.inf, n) for i in range(len(deltas))]
for i in range(6):
    if p_values[i] < alpha:
        print(f'b{i} значим')
    else:
        print(f'b{i} НЕ значим')

b0 значим
b1 значим
b2 значим
b3 значим
b4 значим
b5 НЕ значим


# c) Определить коэффициент детерминации

In [152]:
def integrate_fisher(a, b, d1, d2):
    def func(x):
        return x**(d1/2-1)*(1-x)**(d2/2-1)
    Beta = scipy.integrate.quad(func, 0, 1)[0]
    def fisher(x):
        return pow(d1/d2, d1/2) * pow(x, (d1/2 - 1)) * pow((1 + d1*x/d2), -(d1+d2)/2) / Beta 
    return scipy.integrate.quad(fisher, a, b)[0]


tss = TSS(y)
r_2 = R_2(tss, rss)
print('Коэффициент детерминации: R^2 =', r_2)

p = ksi.shape[1]
delta_r_2 = r_2*(n-p)/((1-r_2)*(p-1))

p_value_r_2 = integrate_fisher(delta_r_2, math.inf, p-1, n-p)
print('p-value =', p_value_r_2)
if p_value_r_2 < alpha:
    print('Коэффициент детерминации значим')
else:
    print('Коэффициент детерминации НЕ значим')


Коэффициент детерминации: R^2 = 0.7159529969544839
p-value = 4.943455361829612e-11
Коэффициент детерминации значим


# d) Найти значение в точке Xk = 0 и построить 95% доверительный интервал

In [153]:
x_0 = np.array([0]*5)
y_0 = np.dot(koefs[1:], x_0) + koefs[0]
print('y =', y_0)

beta = 0.95
ys = []
N = 10000
for i in range(N):
    ys.append(scipy.stats.norm.rvs(loc=2, scale=1.5, size=1)[0])
ys.sort()
print('Доверительный интервал по параметрическому бутстрапу:', end=' ')
print(f'[{ys[int(N*(1-beta)/2)]}, {ys[int(N*(1+beta)/2)]}]')


y = 1.661881335371115
Доверительный интервал по параметрическому бутстрапу: [-0.8952884209494645, 4.9299223669707795]


# e) Проверить предположение о независимости ошибок измерения

In [154]:
def integrate_norm(a, b, m, G):
    def norm(x):
        return pow(math.e, -(x-m)**2/(2*G**2))/(G*math.sqrt(2*math.pi))
    return scipy.integrate.quad(norm, a, b)[0]

I = 0
for i in range(n-1):
    for j in range(i+1, n):
        if e[i] > e[j]:
            I += 1
    
delta_invers = (I-n*(n-1)/4)/math.sqrt(n**3/36)
p_value_invers = 2*integrate_norm(abs(delta_invers), math.inf, 0, 1)
print('p-value =', p_value_invers)
if p_value_invers < alpha:
    print('Предположение НЕверное')
else:
    print('Нет оснований отвергнуть предположение')
        

p-value = 0.014880444769909915
Предположение НЕверное


# f) Проверить предположение о нормальности распределения ошибок

In [155]:
def Ak_ass(k, sample):
    res = 0
    for i in range(len(sample)):
        res += sample[i]**k
    return res/n
        
def OMM(sample):
    t1 = Ak_ass(1, sample)
    t2 = math.sqrt(Ak_ass(2, sample) - t1**2)
    return [t1, t2]

teta1, teta2 = OMM(e) 
print('teta1:',teta1, '\nteta2:', teta2)

F = [1/n]
for i in range(1, n):
    F.append(F[-1] + 1/n)
    

err = [e[i] for i in range(n)]
err.sort()

x_norm = [err[i] for i in range(n)]
y_norm = [integrate_norm(-math.inf, err[0], teta1, teta2)]
for i in range(1, n):
    y_norm.append(y_norm[-1] + integrate_norm(err[i-1], err[i], teta1, teta2))
    
delta_kalm = math.sqrt(n)*max([abs(F[i]-y_norm[i]) for i in range(n)]+[abs(F[i-1]-y_norm[i]) for i in range(1,n)])

N = 10000

deltas = []
for i in range(N):
    tmp_err = scipy.stats.norm.rvs(loc=teta1, scale=teta2, size=n)
    t1, t2 = OMM(tmp_err)
    #print(t1, t2)
    tmp_err.sort()
    tmp_y_norm = [integrate_norm(-math.inf, tmp_err[0], t1, t2)]
    for i in range(1, n):
        tmp_y_norm.append(tmp_y_norm[-1] + integrate_norm(tmp_err[i-1], tmp_err[i], t1, t2))
    a = max([abs(F[i]-tmp_y_norm[i]) for i in range(n)])
    b = max([abs(F[i-1]-tmp_y_norm[i]) for i in range(1,n)])
    deltas.append(math.sqrt(n)*max(a, b))

deltas.sort()

for i in range(N-1):
    if deltas[i+1] >= delta_kalm > deltas[i]:
        print('p-value:', 1-(i-1)/N)
        if 1-(i-1)/N > alpha:
            print('Не можем отвергнуть предположение')
        else:
            print('Отвергаем гипотезу')
        break

teta1: 2.6645352591003756e-16 
teta2: 1.3527203713926548
p-value: 0.2178
Не можем отвергнуть предположение


# g) Исследовать регрессию на выбросы

In [156]:
e_abs = [abs(e[i]) for i in range(n)]
e_abs.sort()
med = e_abs[int(n/2)]
sigma = med/0.675
emissions = []
for i in range(n):
    if -2*sigma < e[i] < 2*sigma:
        continue
    else:
        emissions.append(i)
if len(emissions) != 0:
    print('Выбросы:')
    for i in range(len(emissions)):
        print(sample[i])
else:
    print('Выбросов нет')

Выбросы:
[-0.06280445587099659, -0.6805754597314742, 0.8265591465473532, 0.2175585503792863, 0.9552378009876286, 1.22720527427437]


# h) Провести кросс-проверку регрессии

In [157]:
def calc_koefs(ksi, y):
    ksi_T = ksi.transpose()
    F = np.matmul(ksi_T, ksi)
    F_inv = np.linalg.inv(F)
    koefs = np.matmul(F_inv, np.matmul(ksi_T, y))
    return koefs

def cross_validation(sample, X):
    ksi = []
    y = []
    for i in range(n-1):
        ksi.append([1] + list(sample[i][:-1]))
        y.append(sample[i][5])
    ksi = np.array(ksi)
    y = np.array(y)
    koefs = calc_koefs(ksi, y)
    return np.dot(koefs[1:], X) + koefs[0]
    
    
sample = np.array(sample)
cv = []
for i in range(n):
    tmp_y = cross_validation(np.delete(sample, i, 0), sample[i][:-1])
    cv.append((sample[i][-1]-tmp_y)**2)
cvss = sum(cv)
r_2_cv = (tss-cvss)/tss
print('R^2   =', r_2)
print('Rcv^2 =', r_2_cv)
    

R^2   = 0.7159529969544839
Rcv^2 = 0.6285892411140521


# i) Проверить адекватность регрессии, сделав 5 повторных измерений в одной точке

In [158]:
K = 5
y_K = [scipy.stats.norm.rvs(loc=0, scale=1.5, size=1)[0] for i in range(K)]
print('5 значений в точке Xk=0:',y_K)

sigma_K_2 = 0
y_K_average = sum(y_K)/K
for i in range(K):
    sigma_K_2 += (y_K[i] - y_K_average)**2
sigma_K_2 /= (K - 1)

if rss <= (K - 1)*sigma_K_2:
    print('Модель адекватна')
else:
    delta = rss/((n-p)*sigma_K_2)
    p_value = integrate_fisher(delta, math.inf, n-p, K-1)
    print('p_value =', p_value)
    if p_value < alpha:
        print('Модель не адекватна')
    else:
        print('Не можем отвергнуть адекватность модели')


5 значений в точке Xk=0: [0.32244681468234415, 1.2230640007348719, 1.1668273584977322, -1.2259450347982033, -2.0154336820258942]
p_value = 0.5916477367093906
Не можем отвергнуть адекватность модели


# j) Удалить переменную, соответствующую наименее значимому коэффициенту и повторить пункты b и c, сравнить уравнения регрессии

In [159]:
print('Удалим X5, так как соответствующий ему коэффициент незначим')

Удалим X5, так как соответствующий ему коэффициент незначим


# j.b)

In [160]:
ksi2 = []
y2 = []
for i in range(n):
    ksi2.append([1] + list(sample[i][:-2]))
    y2.append(sample[i][5])
ksi2 = np.array(ksi2)
y2 = np.array(y2)
koefs2 = calc_koefs(ksi2, y2)

print('Коэффициенты: \n')
for i in range(len(koefs2)):
    print(f'b{i} = {koefs2[i]}')

Коэффициенты: 

b0 = 1.6759731565120357
b1 = 2.974270009067486
b2 = -1.757747080130401
b3 = 0.9792058175724779
b4 = 0.9608402914251825


In [161]:
e2 = calc_e(ksi2, y2, koefs2)
rss2 = RSS(e2)
    
deltas2 = [calc_delta(ksi2, koefs2, i, rss2) for i in range(5)]

p_values2 = [2*integrate_student(abs(deltas2[i]), math.inf, n) for i in range(len(deltas2))]
for i in range(5):
    if p_values[i] < alpha:
        print(f'b{i} значим')
    else:
        print(f'b{i} НЕ значим')

b0 значим
b1 значим
b2 значим
b3 значим
b4 значим


# j.c)

In [162]:
tss2 = TSS(y)
r_22 = R_2(tss, rss)
print('Коэффициент детерминации: R^2 =', r_22)

p2 = ksi2.shape[1]
delta_r_22 = r_2*(n-p2)/((1-r_22)*(p2-1))

p_value_r_22 = integrate_fisher(delta_r_22, math.inf, p2-1, n-p2)
print('p-value =', p_value_r_22)
if p_value_r_22 < alpha:
    print('Коэффициент детерминации значим')
else:
    print('Коэффициент детерминации НЕ значим')

Коэффициент детерминации: R^2 = 0.7159529969544839
p-value = 8.600087976801329e-12
Коэффициент детерминации значим


# Сравнение уравнений регрессии

In [212]:
delta_comp = (rss2-rss)*(n-p)/(rss*(p-p2))

p_value_comp = integrate_fisher(delta_comp, math.inf, p-p2, n-p2)
print('p-value =', p_value_comp)
if p_value_comp < alpha:
    print('Отвергаем новое уравнение регрессии')
else:
    print('Не можем отвергнуть новое уравнение регрессии')


p-value = 0.22131155875140704
Не можем отвергнуть новое уравнение регрессии


# k) Сравнить уравнения регрессии бутстрапом

In [211]:

import random

N = 1000
sample_T = sample.transpose()
deltas_comp = []
for u in range(N):
    tmp_sample = []
    for l in range(6):
        line = [random.choice(sample_T[l]) for j in range(n)]
        tmp_sample.append(line)
    tmp_sample = np.array(tmp_sample).transpose()
        
    
    tmp_ksi = []
    tmp_y = []
    for i in range(n):
        tmp_ksi.append([1] + list(tmp_sample[i][:-1]))
        tmp_y.append(tmp_sample[i][5])
    tmp_ksi = np.array(tmp_ksi)
    tmp_y = np.array(tmp_y)
    tmp_koefs = calc_koefs(tmp_ksi, tmp_y)
    tmp_e = calc_e(tmp_ksi, tmp_y, tmp_koefs)
    tmp_rss = RSS(tmp_e)
    
    tmp_ksi2 = []
    tmp_y2 = []
    for i in range(n):
        tmp_ksi2.append([1] + list(tmp_sample[i][:-2]))
        tmp_y2.append(tmp_sample[i][5])
    tmp_ksi2 = np.array(tmp_ksi2)
    tmp_y2 = np.array(tmp_y2)
    tmp_koefs2 = calc_koefs(tmp_ksi2, tmp_y2)
    tmp_e2 = calc_e(tmp_ksi2, tmp_y2, tmp_koefs2)
    tmp_rss2 = RSS(tmp_e2)
    
    deltas_comp.append((tmp_rss2-tmp_rss)*(n-p)/(tmp_rss*(p-p2)))

deltas_comp.sort()


for i in range(N-1):
    if deltas_comp[i+1] >= delta_comp > deltas_comp[i]:
        print('p-value:', 1-(i-1)/N)
        if 1-(i-1)/N < alpha:
            print('Отвергаем новое уравнение регрессии')
        else:
            print('Не можем отвергнуть новое уравнение регрессии')
    
    


p-value: 0.21299999999999997
Не можем отвергнуть новое уравнение регрессии
